<a href="https://colab.research.google.com/github/forminju/DACON_anomaly_detection_/blob/main/0227_1_2___smartfactory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import random
import time
import pickle
import psutil
import gc


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from umap import UMAP

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold

# setting some globl config

plt.style.use('ggplot')
orange_black = [
    '#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820'
]
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams["figure.facecolor"] = '#FFFACD'
plt.rcParams["axes.facecolor"] = '#FFFFE0'
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = orange_black[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'


import warnings
warnings.filterwarnings("ignore")

INFERENCE = True

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [4]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(42)

In [5]:
%cd /content/drive/MyDrive/스마트공장

/content/drive/MyDrive/스마트공장


# Data Load

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/스마트공장/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/스마트공장/test.csv')

In [7]:
train_x = train_df.drop(columns=['TIMESTAMP', 'Y_Class','Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['TIMESTAMP'])

# 파생변수 생성

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
train_x

,PRODUCT_ID,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,TRAIN_595,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,TRAIN_596,T100304,O_31,40.0,94.0,0.0,45.0,11.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
train_x['LINE_PRODUCT'] = train_x['LINE'].str.cat(train_x['PRODUCT_CODE'])
test_x['LINE_PRODUCT'] = test_x['LINE'].str.cat(test_x['PRODUCT_CODE'])

In [11]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE', 'LINE_PRODUCT']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')
     

Done.


In [12]:
train_x.drop(['LINE', 'PRODUCT_CODE'],axis=1, inplace=True)
test_x.drop(['LINE', 'PRODUCT_CODE'],axis=1, inplace=True)

# Imputation 기법 적용

In [13]:
train_x = train_x.dropna(how='all')
test_x = test_x[train_x.columns]

In [14]:
test_x = test_x[train_x.columns]

In [15]:
train_x = train_x.dropna(how='all',axis=1)
test_x = test_x[train_x.columns]

In [16]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=50)

imputer_output = imputer.fit_transform(train_x.iloc[:,1:])

In [17]:
train_x.iloc[:,1:] = pd.DataFrame(imputer_output, columns=train_x.iloc[:,1:].columns, index=list(train_x.iloc[:,1:].index.values));train_x

,PRODUCT_ID,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,LINE_PRODUCT
0,TRAIN_000,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,368.296296,353.00,39.3400,40.8900,32.5600,34.0900,77.770,1.0,2.0
1,TRAIN_001,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,367.735849,353.00,38.8900,42.8200,43.9200,35.3400,72.550,1.0,3.0
2,TRAIN_002,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,383.00,367.320755,353.00,39.1900,36.6500,42.4700,36.5300,78.350,1.0,2.0
3,TRAIN_003,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,384.00,369.188679,353.00,37.7400,39.1700,52.1700,30.5800,71.780,1.0,3.0
4,TRAIN_004,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,383.00,367.351852,352.00,38.7000,41.8900,46.9300,33.0900,76.970,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,2.00,95.00,0.0,45.0,10.00,0.0,50.00,10.00,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.446,1.0,7.0
594,TRAIN_594,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,384.00,369.811321,353.00,49.4700,53.0700,50.8900,55.1000,66.490,1.0,2.0
595,TRAIN_595,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,367.018868,352.00,47.1100,49.5134,49.2178,47.3692,68.439,1.0,2.0
596,TRAIN_596,40.00,94.00,0.0,45.0,11.00,0.0,45.00,10.00,31.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.446,1.0,4.0


In [18]:
test_x = test_x[train_x.columns]

In [19]:
imputer_output_test = imputer.transform(test_x.iloc[:,1:])

In [20]:
test_x.iloc[:,1:] = pd.DataFrame(imputer_output_test, columns=test_x.iloc[:,1:].columns, index=list(test_x.iloc[:,1:].index.values));test_x

,PRODUCT_ID,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,LINE_PRODUCT
0,TEST_000,2.00,94.00,0.0,45.0,10.00,0.0,51.0,10.0,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,7.0
1,TEST_001,2.00,93.00,0.0,45.0,11.00,0.0,45.0,10.0,31.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,5.0
2,TEST_002,2.00,95.00,0.0,45.0,11.00,0.0,45.0,10.0,31.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,5.0
3,TEST_003,4.78,98.46,0.0,45.0,10.84,0.0,45.0,10.0,31.0,...,467.00,444.192308,423.00,46.9280,49.4274,49.0986,47.4816,68.4868,1.0,0.0
4,TEST_004,4.78,98.46,0.0,45.0,10.84,0.0,45.0,10.0,31.0,...,465.00,443.211539,423.00,46.9280,49.4274,49.0986,47.4816,68.4868,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2.00,91.00,0.0,45.0,10.00,0.0,51.0,10.0,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,7.0
306,TEST_306,2.00,96.00,0.0,45.0,11.00,0.0,45.0,10.0,31.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,5.0
307,TEST_307,2.00,91.00,0.0,45.0,10.00,0.0,50.0,10.0,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,7.0
308,TEST_308,2.00,95.00,0.0,45.0,10.00,0.0,51.0,10.0,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.4460,1.0,7.0


In [21]:
train_x

,PRODUCT_ID,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,LINE_PRODUCT
0,TRAIN_000,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,368.296296,353.00,39.3400,40.8900,32.5600,34.0900,77.770,1.0,2.0
1,TRAIN_001,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,367.735849,353.00,38.8900,42.8200,43.9200,35.3400,72.550,1.0,3.0
2,TRAIN_002,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,383.00,367.320755,353.00,39.1900,36.6500,42.4700,36.5300,78.350,1.0,2.0
3,TRAIN_003,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,384.00,369.188679,353.00,37.7400,39.1700,52.1700,30.5800,71.780,1.0,3.0
4,TRAIN_004,2.58,97.88,0.0,45.0,10.60,0.0,45.00,10.00,31.0,...,383.00,367.351852,352.00,38.7000,41.8900,46.9300,33.0900,76.970,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,2.00,95.00,0.0,45.0,10.00,0.0,50.00,10.00,52.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.446,1.0,7.0
594,TRAIN_594,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,384.00,369.811321,353.00,49.4700,53.0700,50.8900,55.1000,66.490,1.0,2.0
595,TRAIN_595,1.92,94.40,0.0,45.0,10.44,0.0,48.26,10.02,41.5,...,383.00,367.018868,352.00,47.1100,49.5134,49.2178,47.3692,68.439,1.0,2.0
596,TRAIN_596,40.00,94.00,0.0,45.0,11.00,0.0,45.00,10.00,31.0,...,383.82,367.359196,350.78,52.1678,55.9140,49.1138,53.4380,65.446,1.0,4.0


# vif 기반 변수 줄이기

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(v):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = v.columns
    vif["VIF"] = [variance_inflation_factor(v.values,i) for i in range(v.shape[1])]

    high_vif = vif[vif["VIF"] > 5].sort_values("VIF",ascending=False)

    
    return(high_vif)

In [23]:
corr_matrix = train_x.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop1 = [column for column in upper.columns if any(upper[column] < -0.9)]
to_drop2 = [column for column in upper.columns if any(upper[column] > 0.9)]
print(to_drop1)
print(to_drop2)
drop=to_drop1+to_drop2

['X_20', 'X_21', 'X_22', 'X_38', 'X_44', 'X_47', 'X_48', 'X_50', 'X_53', 'X_63', 'X_65', 'X_86', 'X_87', 'X_88', 'X_97', 'X_98', 'X_99', 'X_101', 'X_103', 'X_117', 'X_119', 'X_120', 'X_121', 'X_123', 'X_148', 'X_149', 'X_150', 'X_242', 'X_243', 'X_265', 'X_266', 'X_267', 'X_295', 'X_303', 'X_306', 'X_320', 'X_321', 'X_331', 'X_332', 'X_333', 'X_364', 'X_365', 'X_366', 'X_367', 'X_368', 'X_370', 'X_480', 'X_512', 'X_551', 'X_568', 'X_685', 'X_781', 'X_854', 'X_855', 'X_910', 'X_912', 'X_960', 'X_961', 'X_962', 'X_963', 'X_964', 'X_965', 'X_978', 'X_979', 'X_981', 'X_982', 'X_983', 'X_1010', 'X_1012', 'X_1018', 'X_1019', 'X_1026', 'X_1027', 'X_1028', 'X_1029', 'X_1030', 'X_1031', 'X_1032', 'X_1034', 'X_1035', 'X_1036', 'X_1037', 'X_1038', 'X_1039', 'X_1042', 'X_1043', 'X_1044', 'X_1045', 'X_1046', 'X_1048', 'X_1049', 'X_1051', 'X_1052', 'X_1053', 'X_1054', 'X_1055', 'X_1056', 'X_1059', 'X_1060', 'X_1061', 'X_1062', 'X_1063', 'X_1065', 'X_1066', 'X_1068', 'X_1069', 'X_1071', 'X_1072', 'X_

In [24]:
len(drop)

2419

In [25]:
train_x = train_x.drop(drop,1)
test_x = test_x.drop(drop,1)

In [26]:
train_x.shape

(598, 1201)

In [27]:
features_train = train_x.copy()
features_test = test_x.copy()

In [28]:
train_x.iloc[:,1:].columns

Index(['X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9', 'X_10',
       ...
       'X_2798', 'X_2799', 'X_2800', 'X_2801', 'X_2837', 'X_2839', 'X_2840',
       'X_2842', 'X_2871', 'LINE_PRODUCT'],
      dtype='object', length=1200)

In [29]:
train_x.shape

(598, 1201)

In [30]:
test_x.shape

(310, 1201)

# MinMaxScaler

In [31]:
from sklearn.preprocessing import MinMaxScaler

In [32]:
scaler = MinMaxScaler()

In [33]:
#train data 변환
X_group_train = scaler.fit_transform(features_train.iloc[:,1:])

#test data 변환
X_group_test = scaler.transform(features_test.iloc[:,1:])

In [34]:
train_x.iloc[:,1:] = pd.DataFrame(X_group_train, columns=train_x.iloc[:,1:].columns, index=list(train_x.iloc[:,1:].index.values));train_x

,PRODUCT_ID,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,...,X_2798,X_2799,X_2800,X_2801,X_2837,X_2839,X_2840,X_2842,X_2871,LINE_PRODUCT
0,TRAIN_000,0.009020,0.493333,0.0,0.0,0.44,0.0,0.191765,0.02,0.5,...,0.00,0.250,0.211356,0.00,0.577143,0.514563,0.910714,0.048394,0.0,0.285714
1,TRAIN_001,0.009020,0.493333,0.0,0.0,0.44,0.0,0.191765,0.02,0.5,...,0.50,0.500,0.748993,1.00,0.280000,0.495146,1.000000,0.236239,0.0,0.428571
2,TRAIN_002,0.015490,0.725333,0.0,0.0,0.60,0.0,0.000000,0.00,0.0,...,0.50,0.500,0.473847,0.50,0.577143,0.514563,0.053571,0.048624,0.0,0.285714
3,TRAIN_003,0.015490,0.725333,0.0,0.0,0.60,0.0,0.000000,0.00,0.0,...,0.50,0.500,0.745712,0.75,0.134286,0.504854,0.964286,0.250000,0.0,0.428571
4,TRAIN_004,0.015490,0.725333,0.0,0.0,0.60,0.0,0.000000,0.00,0.0,...,0.50,0.500,0.483221,0.50,0.577143,0.524272,0.053571,0.049083,0.0,0.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,0.009804,0.533333,0.0,0.0,0.00,0.0,0.294118,0.00,1.0,...,0.54,0.615,0.621319,0.58,0.339429,0.455728,0.824286,0.141440,0.0,1.000000
594,TRAIN_594,0.009020,0.493333,0.0,0.0,0.44,0.0,0.191765,0.02,0.5,...,0.50,0.750,0.600740,0.50,0.425714,0.485437,0.910714,0.000917,0.0,0.285714
595,TRAIN_595,0.009020,0.493333,0.0,0.0,0.44,0.0,0.191765,0.02,0.5,...,0.50,0.750,0.582660,0.50,0.562857,0.495146,0.910714,0.017202,0.0,0.285714
596,TRAIN_596,0.382353,0.466667,0.0,0.0,1.00,0.0,0.000000,0.00,0.0,...,0.54,0.615,0.621319,0.58,0.339429,0.455728,0.824286,0.141440,0.0,0.571429


In [35]:
test_x.iloc[:,1:] = pd.DataFrame(X_group_test, columns=test_x.iloc[:,1:].columns, index=list(test_x.iloc[:,1:].index.values))

# 분류기

In [36]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from collections import Counter

In [37]:
classifiers = [['Naive Bayes :', GaussianNB()],
               ['KNeighbours :', KNeighborsClassifier()],
               ['SVM :', SVC()],
               ['LogisticRegression :', LogisticRegression()],
               ['DecisionTree :',DecisionTreeClassifier()],
               ['RandomForest :',RandomForestClassifier()],
               ['LGBMClassifier:', LGBMClassifier()],
               ['XGBClassifier: ', XGBClassifier()]]

In [38]:
from imblearn.over_sampling import SMOTE

osmote=SMOTE()
Xs_train,ys_train=osmote.fit_resample(train_x.iloc[:,1:],train_y)

print(Counter(ys_train))

Counter({1: 407, 2: 407, 0: 407})


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
Xs_train, Xs_valid, ys_train, ys_valid = train_test_split(Xs_train, ys_train, test_size=0.2, random_state=42)

In [41]:
for name,classifier in classifiers:
    clf=classifier.fit(Xs_train,ys_train)
    y_pred=classifier.predict(Xs_valid)
    print(f'\n {name} \n')
    print(f'Training Score for {name}  {clf.score(Xs_train,ys_train) * 100:.2f}' )
    print(f'Testing Score for {name} {clf.score(Xs_valid,ys_valid) * 100:.2f}' )
    print(f'Classification report  \n {classification_report(ys_valid,y_pred)}' )
    print(f'Confusion matrix  \n {confusion_matrix(ys_valid,y_pred)}' )
    #print(f'ROC AUC  : {roc_auc_score(valid_y,y_pred)}' )


 Naive Bayes : 

Training Score for Naive Bayes :  63.93
Testing Score for Naive Bayes : 62.04
Classification report  
               precision    recall  f1-score   support

           0       0.91      0.49      0.64        83
           1       0.51      0.73      0.60        82
           2       0.61      0.64      0.63        80

    accuracy                           0.62       245
   macro avg       0.68      0.62      0.62       245
weighted avg       0.68      0.62      0.62       245

Confusion matrix  
 [[41 29 13]
 [ 3 60 19]
 [ 1 28 51]]

 KNeighbours : 

Training Score for KNeighbours :  75.51
Testing Score for KNeighbours : 68.57
Classification report  
               precision    recall  f1-score   support

           0       0.68      0.99      0.80        83
           1       0.89      0.10      0.18        82
           2       0.68      0.97      0.80        80

    accuracy                           0.69       245
   macro avg       0.75      0.69      0.59     

In [42]:
from imblearn.combine import SMOTETomek
rus=SMOTETomek()
Xrus_train,yrus_train =rus.fit_resample (train_x.iloc[:,1:],train_y)

print(Counter(yrus_train))

Counter({1: 407, 2: 407, 0: 407})


In [43]:
Xrus_train, Xrus_valid, yrus_train, yrus_valid = train_test_split(Xrus_train, yrus_train, test_size=0.2, random_state=42)

In [44]:
for name,classifier in classifiers:
    clf=classifier.fit(Xrus_train,yrus_train)
    y_pred=classifier.predict(Xrus_valid)
    print(f'\n {name} \n')
    print(f'Training Score for {name}  {clf.score(Xrus_train,yrus_train) * 100:.2f}' )
    print(f'Testing Score for {name} {clf.score(Xrus_valid,yrus_valid) * 100:.2f}' )
    print(f'Classification report  \n {classification_report(yrus_valid,y_pred)}' )
    print(f'Confusion matrix  \n {confusion_matrix(yrus_valid,y_pred)}' )
    #print(f'ROC AUC  : {roc_auc_score(valid_y,y_pred)}' )


 Naive Bayes : 

Training Score for Naive Bayes :  63.52
Testing Score for Naive Bayes : 60.82
Classification report  
               precision    recall  f1-score   support

           0       0.87      0.47      0.61        83
           1       0.51      0.73      0.60        82
           2       0.61      0.62      0.62        80

    accuracy                           0.61       245
   macro avg       0.66      0.61      0.61       245
weighted avg       0.66      0.61      0.61       245

Confusion matrix  
 [[39 30 14]
 [ 4 60 18]
 [ 2 28 50]]

 KNeighbours : 

Training Score for KNeighbours :  72.85
Testing Score for KNeighbours : 66.53
Classification report  
               precision    recall  f1-score   support

           0       0.65      1.00      0.79        83
           1       1.00      0.05      0.09        82
           2       0.67      0.95      0.79        80

    accuracy                           0.67       245
   macro avg       0.77      0.67      0.56     

In [45]:
from imblearn.over_sampling import BorderlineSMOTE

cc=BorderlineSMOTE()
Xcc_train,ycc_train =cc.fit_resample (train_x.iloc[:,1:],train_y)

print(Counter(ycc_train))

Counter({1: 407, 2: 407, 0: 407})


In [46]:
Xcc_train, Xcc_valid, ycc_train, ycc_valid = train_test_split(Xcc_train, ycc_train, test_size=0.2, random_state=42)

In [47]:
for name,classifier in classifiers:
    clf=classifier.fit(Xcc_train,ycc_train)
    y_pred=classifier.predict(Xcc_valid)
    print(f'\n {name} \n')
    print(f'Training Score for {name}  {clf.score(Xcc_train,ycc_train) * 100:.2f}' )
    print(f'Testing Score for {name} {clf.score(Xcc_valid,ycc_valid) * 100:.2f}' )
    print(f'Classification report  \n {classification_report(ycc_valid,y_pred)}' )
    print(f'Confusion matrix  \n {confusion_matrix(ycc_valid,y_pred)}' )


 Naive Bayes : 

Training Score for Naive Bayes :  64.34
Testing Score for Naive Bayes : 57.55
Classification report  
               precision    recall  f1-score   support

           0       0.84      0.46      0.59        83
           1       0.48      0.73      0.58        82
           2       0.58      0.54      0.56        80

    accuracy                           0.58       245
   macro avg       0.63      0.58      0.58       245
weighted avg       0.64      0.58      0.58       245

Confusion matrix  
 [[38 30 15]
 [ 6 60 16]
 [ 1 36 43]]

 KNeighbours : 

Training Score for KNeighbours :  75.31
Testing Score for KNeighbours : 69.39
Classification report  
               precision    recall  f1-score   support

           0       0.68      0.99      0.80        83
           1       0.86      0.15      0.25        82
           2       0.69      0.95      0.80        80

    accuracy                           0.69       245
   macro avg       0.74      0.69      0.62     

# sMOTETOMEK + votingclassifier

In [48]:
from imblearn.combine import SMOTETomek
rus=SMOTETomek()
Xrus_train,yrus_train =rus.fit_resample (train_x.iloc[:,1:],train_y)

print(Counter(yrus_train))

Counter({1: 407, 2: 407, 0: 407})


# 베이지안 최적화

In [49]:
!pip install bayesian-optimization

from bayes_opt import BayesianOptimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
from sklearn.metrics import f1_score
rf_params={
    'max_depth':(1,150),
    'n_estimators':(10,100)
}

def rf_bo(max_depth, n_estimators):
  params={
      'max_depth':int(round(max_depth)),
      'n_estimators':int(round(n_estimators))
  }

  model=RandomForestClassifier( **params,  n_jobs=-1, random_state=42)
  
  X_train,X_valid,y_train,y_valid=train_test_split(Xcc_train.iloc[:,1:],ycc_train,test_size=0.8)
  model.fit(X_train, y_train)

  score=f1_score(y_valid, model.predict(X_valid), average = 'micro')
  return score

In [68]:
BO_rf = BayesianOptimization(f=rf_bo,pbounds=rf_params,random_state=3,verbose=2)

BO_rf.maximize(init_points=5, n_iter=100)

rf_max_params=BO_rf.max['params']
rf_max_params['max_depth']=int(rf_max_params['max_depth'])
rf_max_params['n_estimators']=int(rf_max_params['n_estimators'])
rf_max_params

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.7196    | 83.07     | 73.73     |
| 2         | 0.7772    | 44.34     | 55.97     |
| 3         | 0.7106    | 134.0     | 90.67     |
| 4         | 0.7554    | 19.71     | 28.65     |
| 5         | 0.6927    | 8.669     | 49.67     |
| 6         | 0.7196    | 45.3      | 55.69     |
| 7         | 0.7862    | 19.66     | 28.57     |
| 8         | 0.7708    | 71.25     | 22.37     |
| 9         | 0.7106    | 19.41     | 28.67     |
| 10        | 0.7734    | 42.57     | 93.02     |
| 11        | 0.7247    | 63.56     | 16.24     |
| 12        | 0.7875    | 42.2      | 93.15     |
| 13        | 0.781     | 141.3     | 94.7      |
| 14        | 0.7452    | 44.59     | 56.15     |
| 15        | 0.7388    | 141.6     | 94.49     |
| 16        | 0.7657    | 42.56     | 93.07     |
| 17        | 0.7657    | 42.42     | 93.03     |
| 18        | 0.8092    | 97.29     | 72.56     |


{'max_depth': 71, 'n_estimators': 64}

In [55]:
xgb_params={
    'gamma':(0,20),
    'max_depth':(1,2000),
    'subsample':(0.5,1),
    'eta' : (0.001, 0.4)
}

In [56]:
def xgb_bo(eta, gamma,max_depth, subsample ):
  params={
      'learning_rate' :max(eta, 0),
      'gamma':int(round(gamma)),
      'max_depth':int(round(max_depth)),
      'subsample':int(round(subsample)),
  }
  model=XGBClassifier(**params, n_jobs=-1, random_state=42)
  X_train,X_valid,y_train,y_valid=train_test_split(Xcc_train.iloc[:,1:],ycc_train,test_size=0.2)

  model.fit(X_train,y_train)

  score=f1_score(y_valid,model.predict(X_valid), average='micro')
  return score

In [57]:
BO_xgb = BayesianOptimization(f=xgb_bo,pbounds=xgb_params,random_state=3,verbose=2)


# Bayesian Optimization을 실행해보세요
BO_xgb.maximize(init_points=5,n_iter=100)
xgb_max_params=BO_xgb.max['params']
xgb_max_params['max_depth']=int(xgb_max_params['max_depth'])
xgb_max_params

|   iter    |  target   |    eta    |   gamma   | max_depth | subsample |
-------------------------------------------------------------------------
| 1         | 0.7959    | 0.2208    | 14.16     | 582.5     | 0.7554    |
| 2         | 0.7857    | 0.3573    | 17.93     | 252.0     | 0.6036    |
| 3         | 0.8418    | 0.02154   | 8.816     | 60.72     | 0.7284    |
| 4         | 0.9082    | 0.26      | 5.57      | 1.353e+03 | 0.7954    |
| 5         | 0.8929    | 0.01057   | 11.18     | 519.2     | 0.7076    |
| 6         | 0.8622    | 0.008341  | 4.807     | 1.354e+03 | 0.8198    |
| 7         | 0.8469    | 0.1609    | 6.579     | 1.354e+03 | 0.9616    |
| 8         | 0.8469    | 0.365     | 9.879     | 518.3     | 0.9369    |
| 9         | 0.8622    | 0.1397    | 4.908     | 1.352e+03 | 0.6803    |
| 10        | 0.8929    | 0.2606    | 4.83      | 1.353e+03 | 0.9735    |
| 11        | 0.852     | 0.1228    | 11.23     | 520.3     | 0.7291    |
| 12        | 0.8316    | 0.1029    | 

{'eta': 0.3423466837492939,
 'gamma': 0.3462431050442949,
 'max_depth': 1351,
 'subsample': 0.7029948366468876}

In [58]:
lgbm_params={
    'n_estimators':(30,100),
    'max_depth':(1,2000),
    'subsample':(0.5,1)
}

In [59]:
def lgbm_bo(n_estimators,max_depth, subsample):
  params={
      'n_estimaotrs':int(round(n_estimators)),
      'max_depth':int(round(max_depth)),
      'subsample':int(round(subsample)),
  }
  model=LGBMClassifier(**params, n_jobs=-1, random_state=42)
  X_train,X_valid,y_train,y_valid=train_test_split(Xcc_train.iloc[:,1:],ycc_train,test_size=0.2)
  model.fit(X_train,y_train)

  score=f1_score(y_valid,model.predict(X_valid),average='micro')
  return score

In [60]:
BO_lgbm = BayesianOptimization(f=lgbm_bo,pbounds=lgbm_params,random_state=3,verbose=2)


# Bayesian Optimization을 실행해보세요
BO_lgbm.maximize(init_points=5,n_iter=100)
lgbm_max_params=BO_lgbm.max['params']
lgbm_max_params['n_estimators']=int(lgbm_max_params['n_estimators'])
lgbm_max_params['max_depth']=int(lgbm_max_params['max_depth'])
lgbm_max_params

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
| 1         | 0.9133    | 1.102e+03 | 79.57     | 0.6455    |
| 2         | 0.949     | 1.022e+03 | 92.51     | 0.9481    |
| 3         | 0.9592    | 252.0     | 44.51     | 0.5257    |
| 4         | 0.9133    | 882.2     | 32.09     | 0.7284    |
| 5         | 0.9541    | 1.299e+03 | 49.49     | 0.8381    |
| 6         | 0.9337    | 251.4     | 42.56     | 0.6701    |
| 7         | 0.9235    | 1.416e+03 | 77.83     | 0.9654    |
| 8         | 0.9439    | 75.94     | 72.97     | 0.9886    |
| 9         | 0.898     | 1.726e+03 | 58.46     | 0.651     |
| 10        | 0.9133    | 1.024e+03 | 92.96     | 0.7399    |
| 11        | 0.9286    | 344.6     | 51.65     | 0.5972    |
| 12        | 0.949     | 251.5     | 42.0      | 0.5055    |
| 13        | 0.9286    | 251.1     | 41.45     | 0.9826    |
| 14        | 0.9694    | 1.541e+03 | 31.59     | 0.7545    |
| 15    

{'max_depth': 1541, 'n_estimators': 31, 'subsample': 0.7545432080303889}

In [69]:
LGBM = LGBMClassifier(**lgbm_max_params)
XGB = XGBClassifier(**xgb_max_params)
RF = RandomForestClassifier(**rf_max_params)
from sklearn.ensemble import VotingClassifier
# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting='soft')

In [70]:
VC.fit(Xcc_train,ycc_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=71,
                                                     n_estimators=64)),
                             ('xgb',
                              XGBClassifier(eta=0.3423466837492939,
                                            gamma=0.3462431050442949,
                                            max_depth=1351,
                                            subsample=0.7029948366468876)),
                             ('lgbm',
                              LGBMClassifier(max_depth=1541, n_estimators=31,
                                             subsample=0.7545432080303889))],
                 voting='soft')

In [71]:
pred=VC.predict(test_x.iloc[:,1:])

In [72]:
submit = pd.read_csv('/content/drive/MyDrive/스마트공장/sample_submission (3).csv')

In [73]:
submit['Y_Class']=pred

In [74]:
submit['Y_Class'].value_counts()

1    265
0     32
2     13
Name: Y_Class, dtype: int64

In [75]:
submit.to_csv("0227_pred2.csv",index=False)